In [5]:
import sys
sys.path.append("..")
from shift_detector import *
from shift_reductor import *
from shift_experiments import *
from shift_tester import *
import pandas as pd
import os
from shift_plot_utils import *
from shift_utils import *
from legacy_utils import *

###  Load data ###

In [9]:
print("Loading data...")  
(X_s_tr, y_s_tr), (X_s_val, y_s_val), (X_t, y_t), orig_dims = import_dataset('los')

Loading data...


### Define Shift Detector and Parameters ###

In [10]:
sign_level = 0.05
dr_technique = "NoRed"
md_test = "Classifier"
red_model = None
datset = "test experiment"
samples = [10, 20, 50, 100, 200, 500, 1000]
sample = samples[5]
orig_dims = X_s_tr.shape[1:]
print("Building shift detector...")
sd = ShiftDetector(dr_technique, md_test, sign_level, red_model, sample, datset)

Building shift detector...


#### Drift from 2019 to 2019 ####

In [11]:
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_tr, y_s_tr, X_s_val, y_s_val, orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.


0.09392214161842637 0.10891152267749686


#### Drift from 2019 to 2020 ####

In [12]:
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_t, y_t, orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.


0.32785030639922275 0.07744927536231884


#### Knockout Shift ####

In [16]:
shift = "ko_shift_1.0"
X_ko, y_ko = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_ko, y_ko, orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.


0.06082786758764797 0.14681967213114755


#### Gaussian Noise Shift ####

In [22]:
shift = "small_gn_shift_0.1"
X_sgn, y_sgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_sgn, y_sgn, orig_dims)
print(p_val,dist)

shift = "large_gn_shift_1.0"
X_lgn, y_lgn = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_lgn, y_lgn, orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.
No GPU detected, fall back on CPU.


0.5731153736169436 0.052812299240158615
0.9747435623461163 0.00981928821685414


#### Multiway Feature Association Shift ####

In [18]:
shift = "mfa_shift_0.5"
X_mfa, y_mfa = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_mfa[:sample,:], y_mfa[:sample], orig_dims)
print(p_val,dist)

No GPU detected, fall back on CPU.


0.7493483684547241 0.04490165631469979


#### Changepoint Shift ####

In [19]:
shift= "cp_shift_0.75"
X_cp, y_cp = apply_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, shift)
p_val, dist, red_dim, red_model, t1_acc, t2_acc = sd.detect_data_shift(X_s_tr, y_s_tr, X_s_val, y_s_val, X_cp[:sample,:], y_cp[:sample], orig_dims)
print(p_val,dist)

Features [15 16 19] are constant.
invalid value encountered in true_divide
No GPU detected, fall back on CPU.


0.788445033272746 0.04038461538461539
